In [1]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip

wget: /shared/centos7/anaconda3/2022.01/lib/libuuid.so.1: no version information available (required by wget)
--2022-04-28 23:37:38--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_image_2.zip
Connecting to 10.99.0.130:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 12569945557 (12G) [application/zip]
Saving to: ‘data_object_image_2.zip’

100%[===================================>] 12,569,945,557 31.9MB/s   in 6m 57s 

2022-04-28 23:44:38 (28.8 MB/s) - ‘data_object_image_2.zip’ saved [12569945557/12569945557]



In [ ]:
!unzip data_object_image_2.zip

In [4]:
!wget https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip

wget: /shared/centos7/anaconda3/2022.01/lib/libuuid.so.1: no version information available (required by wget)
--2022-04-28 23:49:29--  https://s3.eu-central-1.amazonaws.com/avg-kitti/data_object_label_2.zip
Connecting to 10.99.0.130:3128... connected.
Proxy request sent, awaiting response... 200 OK
Length: 5601213 (5.3M) [application/zip]
Saving to: ‘data_object_label_2.zip’

100%[======================================>] 5,601,213   8.13MB/s   in 0.7s   

2022-04-28 23:49:31 (8.13 MB/s) - ‘data_object_label_2.zip’ saved [5601213/5601213]



In [ ]:
!unzip data_object_label_2.zip

In [2]:
import numpy as np
import os
from keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from keras.optimizers import adam_v2
from loss_func import orientation_loss
from preprocess_data import load_and_process_annotation_data,train_data_gen
from model import bbox_3D_net

In [12]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [13]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [14]:
model = bbox_3D_net((224,224,3),bin_num=6,vgg_weights='imagenet')

2022-04-29 00:05:46.454079: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-29 00:05:47.370465: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 10785 MB memory:  -> device: 0, name: Tesla K40m, pci bus id: 0000:0b:00.0, compute capability: 3.5


94781440/94765736 [==============================] - 1s 0us/step
Instructions for updating:
dim is deprecated, use axis instead


In [15]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [16]:
minimizer = adam_v2.Adam(lr=1e-5)

early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=10, mode='min', verbose=1)
checkpoint = ModelCheckpoint('weights.hdf5', monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
tensorboard = TensorBoard(log_dir='../logs/', histogram_freq=0, write_graph=True, write_images=False)

/shared/centos7/anaconda3/2022.01/envs/ALY6980_GPU_01_2022/lib/python3.7/site-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [17]:
model.compile(optimizer=minimizer,#minimizer,
              loss={'dimension': 'mean_squared_error', 'orientation': orientation_loss, 'confidence': 'categorical_crossentropy'},
              loss_weights={'dimension': 2., 'orientation': 1., 'confidence': 4.},
              metrics={'dimension': 'mse', 'orientation': 'mse', 'confidence': 'accuracy'}
             )

In [18]:
label_dir = 'training/label_2/'
image_dir = 'training/image_2/'

In [19]:
classes = [line.strip() for line in open(r'voc_labels.txt').readlines()]
cls_to_ind = {cls:i for i,cls in enumerate(classes)}

In [20]:
cls_to_ind

{'Car': 0,
 'Van': 1,
 'Truck': 2,
 'Pedestrian': 3,
 'Person_sitting': 4,
 'Cyclist': 5,
 'Tram': 6}

In [21]:
dims_avg = np.loadtxt(r'voc_dims.txt',delimiter=',')

In [22]:
objs = load_and_process_annotation_data(label_dir, dims_avg, cls_to_ind)

In [23]:
objs_num = len(objs)
train_num = int(0.9*objs_num)
batch_size = 64
np.random.shuffle(objs)

In [24]:
train_gen = train_data_gen(objs[:train_num], image_dir, batch_size, bin_num=6)

In [25]:
valid_gen = train_data_gen(objs[train_num:], image_dir, batch_size, bin_num=6)

In [26]:
train_epoch_num = int(np.ceil((train_num)/batch_size))
valid_epoch_num = int(np.ceil((objs_num - train_num)/batch_size))

In [33]:
model.fit_generator(generator = train_gen,
                    steps_per_epoch = train_epoch_num,
                    epochs = 30,
                    verbose = 1,
                    validation_data = valid_gen,
                    validation_steps = valid_epoch_num,
                    callbacks = [early_stop, checkpoint],
                    max_queue_size = 3)

/shared/centos7/anaconda3/2022.01/envs/ALY6980_GPU_01_2022/lib/python3.7/site-packages/ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/30
374/374 [==============================] - ETA: 0s - loss: 3.6861 - dimension_loss: 0.4149 - orientation_loss: -0.9576 - confidence_loss: 0.9535 - dimension_mse: 0.4149 - orientation_mse: 0.4217 - confidence_accuracy: 0.6872
Epoch 00001: val_loss improved from inf to 1.60459, saving model to weights.hdf5


/shared/centos7/anaconda3/2022.01/envs/ALY6980_GPU_01_2022/lib/python3.7/site-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


374/374 [==============================] - 664s 2s/step - loss: 3.6861 - dimension_loss: 0.4149 - orientation_loss: -0.9576 - confidence_loss: 0.9535 - dimension_mse: 0.4149 - orientation_mse: 0.4217 - confidence_accuracy: 0.6872 - val_loss: 1.6046 - val_dimension_loss: 0.2564 - val_orientation_loss: -0.9716 - val_confidence_loss: 0.5158 - val_dimension_mse: 0.2564 - val_orientation_mse: 0.4193 - val_confidence_accuracy: 0.8135
Epoch 2/30
374/374 [==============================] - ETA: 0s - loss: 1.3126 - dimension_loss: 0.2431 - orientation_loss: -0.9726 - confidence_loss: 0.4498 - dimension_mse: 0.2431 - orientation_mse: 0.4190 - confidence_accuracy: 0.8397
Epoch 00002: val_loss improved from 1.60459 to 1.07304, saving model to weights.hdf5
374/374 [==============================] - 695s 2s/step - loss: 1.3126 - dimension_loss: 0.2431 - orientation_loss: -0.9726 - confidence_loss: 0.4498 - dimension_mse: 0.2431 - orientation_mse: 0.4190 - confidence_accuracy: 0.8397 - val_loss: 1.073

In [34]:
model.save_weights(r'weights.h5')